<a href="https://colab.research.google.com/github/yasamanAkhoundzadeh/OSLab-Fall2022/blob/main/Detecting_IOT_BotNet_Atacks_On_N_BaIoT_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports And Initializations**

In [ ]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import recall_score, f1_score
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import confusion_matrix, recall_score, classification_report, f1_score
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn import svm

In [ ]:
from google.colab import drive
import os

In [ ]:
drive.mount('/content/drive')
dataPath="/content/drive/MyDrive/University/projects/N-BaIoT/Colab Notebooks/Dataset"
#dataPath="/content/drive/MyDrive/Colab Notebooks/Dataset"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive/University/projects/N-BaIoT/Colab Notebooks/Dataset"


1.benign.csv	      4.gafgyt.tcp.csv	    7.gafgyt.scan.csv
1.gafgyt.combo.csv    4.gafgyt.udp.csv	    7.gafgyt.tcp.csv
1.gafgyt.junk.csv     4.mirai.ack.csv	    7.gafgyt.udp.csv
1.gafgyt.scan.csv     4.mirai.scan.csv	    8.benign.csv
1.gafgyt.tcp.csv      4.mirai.syn.csv	    8.gafgyt.combo.csv
1.gafgyt.udp.csv      4.mirai.udp.csv	    8.gafgyt.junk.csv
1.mirai.ack.csv       4.mirai.udpplain.csv  8.gafgyt.scan.csv
1.mirai.scan.csv      5.benign.csv	    8.gafgyt.tcp.csv
1.mirai.syn.csv       5.gafgyt.combo.csv    8.gafgyt.udp.csv
1.mirai.udp.csv       5.gafgyt.junk.csv     8.mirai.ack.csv
1.mirai.udpplain.csv  5.gafgyt.scan.csv     8.mirai.scan.csv
2.benign.csv	      5.gafgyt.tcp.csv	    8.mirai.syn.csv
2.gafgyt.combo.csv    5.gafgyt.udp.csv	    8.mirai.udp.csv
2.gafgyt.junk.csv     5.mirai.ack.csv	    8.mirai.udpplain.csv
2.gafgyt.scan.csv     5.mirai.scan.csv	    9.benign.csv
2.gafgyt.tcp.csv      5.mirai.syn.csv	    9.gafgyt.combo.csv
2.gafgyt.udp.csv      5.mirai.udp.csv	    9.gafgyt.

In [ ]:
def trainFunction(x_train_data, y_train_data):
    print("\n--- trainFunction is running ---\n")

    fold = KFold(len(x_train_data), 5, shuffle=False)

    c_params = [0.01, 0.1, 1, 10, 100]
    result_tables = pd.DataFrame(columns=['C_parameter', 'Mean recall score'])
    result_tables['C_parameter'] = c_params

    j = 0
    for c_param in c_params:
        print('-------------------------------------------')
        print('C_parameters are equal to:', c_param)
        print('-------------------------------------------\n')

        recall_list = []
        for iteration, indices in enumerate(fold, start=1):
            nb = GaussianNB()
            nb.fit(x_train_data.iloc[indices[0], :], y_train_data.iloc[indices[0], :].values.ravel())

            y_undersample_pred = nb.predict(x_train_data.iloc[indices[1], :].values)
            recall = recall_score(y_train_data.iloc[indices[1], :].values, y_undersample_pred)

            f1_score_value = f1_score(y_train_data.iloc[indices[1], :].values, y_undersample_pred)
            recall_list.append(recall)
            print("Iteration", iteration, "Recall:", recall, "f1_score_value:", f1_score_value)

        print('-------------------------------------------')
        print('Average Recall:', np.mean(recall_list))
        print('-------------------------------------------')
        result_tables.loc[j, 'Mean recall score'] = np.mean(recall_list)
        j = j + 1

    result_tables['Mean recall score'] = result_tables['Mean recall score'].astype('float64')
    best_c_param = result_tables.loc[result_tables['Mean recall score'].idxmax(), 'C_parameter']

    print('---------------------------------------------')
    print('Best C_parameter rate for the model equals to:', best_c_param)
    print('---------------------------------------------')

    return best_c_param



In [ ]:
def showData(data):
    #print(data.shape)
    print(data.head())

In [ ]:
9# Using the virtual server to run the pprogram with all datasets.

print("The maximum value could be 9!\n")
first = int(input("First number for the loop to set the dataset range: "))
last = int(input("Last number for the loop to set the dataset range: "))

# Defining all lists before Going to prepare them and use them in the LOOP.

bng = []
m_udp = []
m_ack = []
m_scan = []
m_syn = []
m_udpplain = []
g_combo = []
g_junk = []
g_scan = []
g_tcp = []
g_udp = []

The maximum value could be 9!

First number for the loop to set the dataset range: 1
Last number for the loop to set the dataset range: 9


In [ ]:
def dataPrepare():

    print("\n--- dataPrepare is running ---\n")
    j = 0
    result = pd.DataFrame()
    for i in range (first, last):

        # benign dataset
        csv_bng = pd.read_csv(f"{dataPath}/{i}.benign.csv")
        csv_bng['Class'] = 0
        #bng.insert(i, csv_bng)


        # gafgyt dataset combo
        csv_g_combo = pd.read_csv(f"{dataPath}/{i}.gafgyt.combo.csv")
        csv_g_combo['Class'] = 1
        #g_combo.insert(i, csv_g_combo)
        # gafgyt dataset junk
        csv_g_junk = pd.read_csv(f"{dataPath}/{i}.gafgyt.junk.csv")
        csv_g_junk['Class'] = 1
        #g_junk.insert(i, csv_g_junk)

        # gafgyt dataset scan
        csv_g_scan = pd.read_csv(f"{dataPath}/{i}.gafgyt.scan.csv")
        csv_g_scan['Class'] = 1
        #g_scan.insert(i, csv_g_scan)

        # gafgyt dataset tcp
        csv_g_tcp = pd.read_csv(f"{dataPath}/{i}.gafgyt.tcp.csv")
        csv_g_tcp['Class'] = 1
        #g_tcp.insert(i, csv_g_tcp)

        # gafgyt dataset udp
        csv_g_udp = pd.read_csv(f"{dataPath}/{i}.gafgyt.udp.csv")
        csv_g_udp['Class'] = 1
        #g_udp.insert(i, csv_g_udp)

        #frames = [bng, m_udp, m_ack, m_scan, m_syn, m_udpplain ,g_combo, g_junk, g_scan, g_tcp, g_udp]
        result = pd.concat([result, csv_bng, csv_g_combo, csv_g_junk, csv_g_scan, csv_g_tcp, csv_g_udp], ignore_index=True)

         # The reason we used while loop here is that we DON'T have any 'mirai' type dataset with number of "3".
        if i != 3 and i!=7:
            # mirai dataset udp
            csv_m_udp = pd.read_csv(f"{dataPath}/{i}.mirai.udp.csv")
            csv_m_udp['Class'] = 1
            #m_udp.insert(i, csv_m_udp)

            # mirai dataset ack
            csv_m_ack = pd.read_csv(f"{dataPath}/{i}.mirai.ack.csv")
            csv_m_ack['Class'] = 1
            #m_ack.insert(i, csv_m_ack)

            # mirai dataset ack
            csv_m_scan = pd.read_csv(f"{dataPath}/{i}.mirai.scan.csv")
            csv_m_scan['Class'] = 1
            #m_scan.insert(i, csv_m_scan)

            # mirai dataset syn
            csv_m_syn = pd.read_csv(f"{dataPath}/{i}.mirai.syn.csv")
            csv_m_syn['Class'] = 1
            #m_syn.insert(i, csv_m_syn)

            # mirai dataset udpplain
            csv_m_udpplain = pd.read_csv(f"{dataPath}/{i}.mirai.udpplain.csv")
            csv_m_udpplain['Class'] = 1
            #m_udpplain.insert(i, csv_m_udpplain)
            # While Loop ends here
            result = pd.concat([result, csv_m_udp, csv_m_ack, csv_m_scan, csv_g_tcp, csv_g_udp], ignore_index=True)

    # concatinate all results in one list
    #result = pd.concat([bng, g_combo, g_junk, g_scan, g_tcp, g_udp], ignore_index=True)

    showData(result)
    return result

In [ ]:
def dataPreprocessing(data):

    print("\n--- dataPreprocessing Function is running ---\n")

    count_class = pd.value_counts(data['Class'],sort=True).sort_index()
    print(count_class)

    print('------')

    X = data.iloc[:, data.columns != 'Class']
    y = data.iloc[:, data.columns == 'Class']

    positive_sample_count = len(data[data.Class == 1])
    print("Positive sample counts equals to：", positive_sample_count)

    negative_sample_index = np.array(data[data.Class == 0].index)
    print("[First Five] Negative sample counts equals to：", negative_sample_index[:5])

    positive_sample_index = data[data.Class == 1].index

    # generate a random sample from the given 1D array with the below code
    # numpy.random.choice(a, size=None, replace=True, p=None)

    random_positive_sample_index = np.random.choice(positive_sample_index, int(1*len(data[data.Class == 0])), replace=False)

    # Adjust the number of samples when detecting anomalies
    # negative_sample_index = np.random.choice(positive_sample_index, len(data[data.Class == 0]), replace=False)

    print("[First Five] Positive sample counts equals to：", random_positive_sample_index[:5])

    under_sample_index = np.concatenate([random_positive_sample_index, negative_sample_index])
    under_sample_data = data.iloc[under_sample_index, :]

    X_under_sample = under_sample_data.iloc[:, under_sample_data.columns != 'Class']
    y_under_sample = under_sample_data.iloc[:, under_sample_data.columns == 'Class']

    print('[After Downsampling] The proportion of +POSITIVE+ samples：', len(under_sample_data[under_sample_data.Class == 1]) / len(under_sample_data))
    print('[After Downsampling] The proportion of -NEGATIVE- samples：', len(under_sample_data[under_sample_data.Class == 0]) / len(under_sample_data))
    print('[After Downsampling] The Length of Under Sample Data: ', len(under_sample_data))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    X_train_under_sample, X_test_under_sample, y_train_under_sample, y_test_under_sample = train_test_split(X_under_sample, y_under_sample, test_size=0.3, random_state=0)


    print('Number of samples in the "Training" set：', len(X_train_under_sample))
    print('Number of samples in the "Test" set：', len(X_test_under_sample))

    return X_train, X_test, y_train, y_test, X_train_under_sample, X_test_under_sample, y_train_under_sample, y_test_under_sample


In [ ]:
def plot_confusion_matrix(confusion_matrix, classes):
    print("\n--- plot_confusion_matrix is running ---\n")

    plt.figure()
    plt.imshow(confusion_matrix, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()

    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    thresh = confusion_matrix.max() / 2.
    range_0 = range(confusion_matrix.shape[0])
    range_1 = range(confusion_matrix.shape[1])

    for i, j in itertools.product(range_0, range_1):
        plt.text(j, i, confusion_matrix[i, j], horizontalalignment="center",
                 color="white" if confusion_matrix[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

    precision = confusion_matrix[1, 1] / (confusion_matrix[1, 1] + confusion_matrix[0, 1])
    recall = confusion_matrix[1, 1] / (confusion_matrix[1, 1] + confusion_matrix[1, 0])
    accuracy = (confusion_matrix[0, 0] + confusion_matrix[1, 1]) / (
                confusion_matrix[0, 0] + confusion_matrix[0, 1] + confusion_matrix[1, 1] + confusion_matrix[1, 0])

    print('Precision rate:', precision)
    print('Recall rate:', recall)
    print('Accuracy:', accuracy)
    print('*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*')

result = dataPrepare()
X_train, X_test, y_train, y_test, X_train_under_sample, X_test_under_sample, y_train_under_sample, y_test_under_sample = dataPreprocessing(result)

# Naive Bayes
nb = GaussianNB()
nb.fit(X_train_under_sample, y_train_under_sample.values.ravel())

y_undersample_pred = nb.predict(X_test_under_sample)

# Set threshold for prediction
threshold = 0.5
y_undersample_pred_binary = [1 if pred >= threshold else 0 for pred in y_undersample_pred]

conf_matrix = confusion_matrix(y_test_under_sample, y_undersample_pred_binary)

np.set_printoptions(precision=2)
class_names = [0, 1]

plot_confusion_matrix(conf_matrix, classes=class_names)

# LightGBM
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

train_data = lgb.Dataset(X_train_under_sample, label=y_train_under_sample)
gbm = lgb.train(params, train_data)

y_undersample_pred = gbm.predict(X_test_under_sample)

# Set threshold for prediction
threshold = 0.5
y_undersample_pred_binary = [1 if pred >= threshold else 0 for pred in y_undersample_pred]

conf_matrix = confusion_matrix(y_test_under_sample, y_undersample_pred_binary)

np.set_printoptions(precision=2)
class_names = [0, 1]

plot_confusion_matrix(conf_matrix, classes=class_names)



--- dataPrepare is running ---

